# Random balanced network example

In the simplest case, a random balanced network consists of two populations of neurons, one of them being excitatory, one being inhibitory. The two populations are connected to themselves and to one another with weights that result in a stable network activity that neither explodes nor dies out. To get the activity of the populations going, each of them gets an  external stimulation. It is interesting to note here that the inhibitory and excitatory neurons themselves do not differ in their parameters, but only in their connection sign and strengths. 

More information about the theory of the random balanced network model and links to further literature can be found on [Pierre Yger's homepage](http://www.yger.net/the-balanced-network/).

The exercise is to fill in the parts below, which are marked "TODO" and create a plot of the network's activity, which roughtly corresponds to the Asynchronous Irregular (AI) state, described on [Pierre's homepage](http://www.yger.net/the-balanced-network/).

Additional information on the commands and models available in NEST can be found on the [NEST documentation index](http://www.nest-simulator.org/documentation/), especially on the [Command index](http://www.nest-simulator.org/helpindex/). Please note that the commands there use SLI's reverse polish notation syntax, which is similar, but not the ame as the one that needs to be used within Python.

In [ ]:
import pylab              # For general plotting
import nest               # the NEST simulator module
import nest.raster_plot   # NEST's raster_plot module

In [ ]:
# simulation parameters
simtime = 1000.           # simulation time (ms)
dt = 0.1                  # simulation resolution (ms)

In [ ]:
# network parameters
NE = 1000                 # number of excitatory neurons
gamma = 0.25              # relative number of inhibitory connections
NI = int(gamma * NE)      # number of inhibitory neurons
epsilon = 0.1             # connection probability (determines fixed indegree)

In [ ]:
# stimulation parameters
nu_ext = 15e3             # external poisson rate

In [ ]:
# neuron paramters
neuron_params = {
    'C_m':     250.0,     # (pF)
    'E_L':       0.0,     # (mV)
    'I_e':       0.0,     # (pA)
    'V_m':       0.0,     # (mV)
    'V_reset':   0.0,     # (mV)
    'V_th':     15.0,     # (mV)
    't_ref':     2.0,     # (ms)
    'tau_m':    10.0,     # (ms)
}

In [ ]:
# synapse paramters
w =  0.1                  # excitatory synaptic weight (mV)
g = -5.0                  # relative inhibitory to excitatory synaptic weight
d =  1.5                  # synaptic transmission delay (ms)

In [ ]:
# reset and configure kernel
nest.ResetKernel()
nest.SetKernelStatus({
    'resolution': dt,    # set simulation resolution
})

# Set default parameters for the neurons and create neuron populations (model: 'iaf_psc_delta')

nest.SetDefaults('iaf_psc_delta', neuron_params)
neurons_e = nest.Create('iaf_psc_delta', NE)
neurons_i = nest.Create('iaf_psc_delta', NI)

# Create a poisson generator and set it's 'rate' to nu_ext

pg = nest.Create('poisson_generator', params={'rate': nu_ext})

# Create spike detectors
spikes_e = nest.Create('spike_detector', params={'withtime': True, 'withgid': True})
spikes_i = nest.Create('spike_detector', params={'withtime': True, 'withgid': True})

# Connect the network

# 1. Excitatory connections
#    Connect the excitatory population to itself and to the inhibitory population using
#    nest.Connect with 'fixed_indegree'. Calculate the indegree from 'NE' and 'epsilon'.

conn_spec_exc = {'rule': 'fixed_indegree', 'indegree': int(epsilon * NE)}
syn_spec_exc = {'delay': d, 'weight': w}
nest.Connect(neurons_e, neurons_e, conn_spec_exc, syn_spec_exc)
nest.Connect(neurons_e, neurons_i, conn_spec_exc, syn_spec_exc)

# 2. Inhibitory connections
#    Connect the inhibitory population to itself and to the excitatory population using
#    nest.Connect with 'fixed_indegree'. Calculate the indegree from 'NI' and 'epsilon'.
#    The weight has to be determines using 

conn_spec_inh = {'rule': 'fixed_indegree', 'indegree': int(epsilon * NE)}
syn_spec_inh = {'delay': d, 'weight': w*g}
nest.Connect(neurons_i, neurons_i, conn_spec_inh, syn_spec_inh)
nest.Connect(neurons_i, neurons_e, conn_spec_inh, syn_spec_inh)

# Connect poisson generator to both populations using the excitatory connection weight

nest.Connect(pg, neurons_e, syn_spec=syn_spec_exc)
nest.Connect(pg, neurons_i, syn_spec=syn_spec_exc)

# Connect the spike detectors to the corresponding populations

nest.Connect(neurons_e, spikes_e)
nest.Connect(neurons_i, spikes_i)

# Run the simulation

nest.Simulate(simtime)

# Calculate the combined mean firing rate of the populations
events_ex = nest.GetStatus(spikes_e, 'n_events')[0]
rate_ex = events_ex / simtime * 1000.0 / NE
events_in = nest.GetStatus(spikes_i, 'n_events')[0]
rate_in = events_in / simtime * 1000.0 / NI
mean_rate = (rate_ex + rate_in) / 2.
print('mean firing rate: ', mean_rate)

# Create a raster plot of the excitatory spiking activity
nest.raster_plot.from_device(spikes_e)
pylab.show()